In [1]:
from collections import defaultdict
import regex
import re

with open('day_19.txt', 'r') as f:
    rules, messages = f.read().split('\n\n')
    
rules = rules.splitlines()
messages = messages.splitlines()

# Part 1

In [2]:
def create_dict(r):
    
    # put the rules in a dictionary wrapped in parentheses
    rule_dict = defaultdict(list)
    nodes = []

    for rule in r:

        key, l = rule.split(': ')

        # keep track of which keys are the terminal nodes for easy access
        if l == '"a"' or l == '"b"':
            nodes.append(key)
            rule_dict[key] = l.replace('"', '')

        else:
            rule_dict[key] = '(' + l + ')'
            
    return(rule_dict, nodes)

In [3]:
def get_regex_rules(r, completed):
    
    # loop over the rules and replace any "completed" rule numbers with their regex representation
    # this starts with replacing 16 and 26 (the terminal nodes) with "b" and "a" whenever we see them in a rule
    
    c_len = 0
    while len(completed) > c_len:

        c_len = len(completed)

        for c in completed:

            for key, val in r.items():

                if key in completed:
                    continue

                # substitute the rule number with its regex representation
                new_val = re.sub(r'\({0} '.format(c), '({} '.format(r[c]), val)
                new_val = re.sub(r' {0} '.format(c), ' {} '.format(r[c]), new_val)
                new_val = re.sub(r' {0}\)'.format(c), ' {})'.format(r[c]), new_val)
                new_val = re.sub(r'\({0}\)'.format(c), '({})'.format(r[c]), new_val)

                # a rule is completed if it contains no numbers, just "a" and "b"
                if not re.search('[0-9]', new_val):
                    completed.append(key)
                    r[key] = new_val.replace(' ', '')

                else:
                    r[key] = new_val
            
    return(r)

In [4]:
rule_dict, completed = create_dict(rules)
regex_rules = get_regex_rules(rule_dict, completed)

In [5]:
# rule 0 is now a regex representation of the full tree and can be used in a match
r = re.compile('^' + regex_rules['0'] + '$')
sum([1 if r.match(i) else 0 for i in messages])

208

# Part 2

In [6]:
rule_dict, completed = create_dict(rules)
regex_rules = get_regex_rules(rule_dict, completed)

In [7]:
# the change to 8 simply means we can repeat rule 42 any number of times
regex_rules['8'] = '({})+'.format(regex_rules['42'])

# the change to rule 11 needs to check recursively for 42(r)*31 where r = 42(r)*31
regex_rules['11'] = r'(({})(?{})*({}))'.format(regex_rules['42'], regex_rules['8'].count('(')+1, regex_rules['31'])

# combine them for final rule
regex_rules['0'] = regex_rules['8'] + regex_rules['11']

In [8]:
r = regex.compile('^' + rule_dict['0'] + '$')
sum([1 if r.match(i) else 0 for i in messages])

316